In [1]:
import sys
sys.path.append('../')

import albumentations as A
import src.training as train
import src.dataset as dataset
from  src.models import resnet,siamesenet
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms

C:\Users\Javier\AppData\Roaming\Python\Python310\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
batch_size = 128

transform = transforms.Compose([
    transforms.Resize((105, 105)),
    transforms.ToTensor()
    ])

albumentations_transform = A.Compose([
    A.ColorJitter(brightness=0.0, contrast=0.0, saturation=0.0, hue=0.25, p=0.0),
    A.GridElasticDeform(num_grid_xy = (7,7), magnitude = 30),
    A.Resize(105,105),
])
same_class_transform = dataset.TorchAlbumentationTransform(albumentations_transform)

train_dataset = dataset.SiameseDataset(
    "../data/cropped_selected_split_data/train",
    transform = transform,
    max_positive_combinations=10,
    max_negative_combinations=10,
    same_class_negatives=0.5,
    same_class_transform=same_class_transform
    )

validation_dataset = dataset.SiameseDataset(
    "../data/cropped_selected_split_data/val",
    transform = transform,
    max_positive_combinations=5,
    max_negative_combinations=5,
    same_class_negatives=0.5,
    same_class_transform=same_class_transform
    )

dataloader = {
    'train': DataLoader(train_dataset, shuffle=True, batch_size=batch_size),
    'val': DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
}

## Model definition

In [ ]:
backbone = resnet.prunnedResnet18(
    pretrained=True,
    freeze_ratio=0.8,
    remove_classifier=True)
model = siamesenet.siameseNet(backbone,512)
result = train.train_siamese_network_V2(model=model,
                                        dataloader=dataloader,
                                        name="siamese_resnet_18",
                                        checkpoint_path="..//checkpoints")

In [ ]:
train.show_graph(result)

In [ ]:
test_dataset = dataset.SiameseDataset(
    "../data/cropped_selected_split_data/test",
    transform = transform,
    max_positive_combinations=5,
    max_negative_combinations=5,
    )
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

train.evaluate_model(test_loader=test_dataloader,
                     model=model)